In [1]:
import tensorflow as tf

import tf_encrypted as tfe
from tf_encrypted.keras import backend as KE
from tf_encrypted.keras.losses import BinaryCrossentropy
from tf_encrypted.keras.optimizers import SGD

from common import DataOwner

import sys
sys.path.append('../')
from utils import print_in_notebook

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'F:\Anaconda3\envs\tfe\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.2.so'


In [2]:
num_features = 10
training_set_size = 2000
test_set_size = 100
batch_size = 100
steps_per_epoch = (training_set_size // batch_size)
epochs = 20

In [3]:
data_owner = DataOwner('data-owner',
                       num_features,
                       training_set_size,
                       test_set_size,
                       batch_size)

x_train, y_train = data_owner.provide_training_data()
x_test, y_test = data_owner.provide_testing_data()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.



In [4]:
@tfe.local_computation('data-owner')
def compute_accuracy(y_pred, y_test):
    correct_prediction = tf.equal(tf.round(y_pred), y_test)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print_op = print_in_notebook(accuracy)
    return print_op

## Define Model

In [5]:
model = tfe.keras.Sequential()
model.add(tfe.keras.layers.Dense(1, batch_input_shape=[batch_size, num_features]))
model.add(tfe.keras.layers.Activation('sigmoid'))

In [6]:
sess = KE.get_session()
sess.run([data_owner.initializer])

model.compile(optimizer=SGD(lr=0.01),
              loss=BinaryCrossentropy())

# Train
model.fit(x_train,
          y_train,
          epochs=epochs,
          steps_per_epoch=steps_per_epoch)

# evaluate
y_pred = model(x_test)
sess.run(compute_accuracy(y_pred, y_test))




Epoch 1/20

2000/2000 [==============================] - 5s 2ms/step - loss: 0.6992
Epoch 2/20
2000/2000 [==============================] - 1s 394us/step - loss: 0.6517
Epoch 3/20
2000/2000 [==============================] - 1s 383us/step - loss: 0.6081
Epoch 4/20
2000/2000 [==============================] - 1s 374us/step - loss: 0.5762
Epoch 5/20
2000/2000 [==============================] - 1s 382us/step - loss: 0.5443
Epoch 6/20
2000/2000 [==============================] - 1s 372us/step - loss: 0.5225
Epoch 7/20
2000/2000 [==============================] - 1s 359us/step - loss: 0.4972
Epoch 8/20
2000/2000 [==============================] - 1s 351us/step - loss: 0.4772 0s - loss
Epoch 9/20
2000/2000 [==============================] - 1s 345us/step - loss: 0.4575
Epoch 10/20
2000/2000 [==============================] - 1s 357us/step - loss: 0.4422
Epoch 11/20
2000/2000 [==============================] - 1s 358us/step - loss: 0.4314
Epoch 12/20
2000/2000 [============================